# 网络构建

神经网络模型是由神经网络层和Tensor操作构成的，`mindspore.nn` 提供了常见神经网络层的实现，在MindSpore中，`Cell`类是构建所有网络的基类，也是网络的基本单元。

下面我们将构建一个用于Mnist数据集分类的神经网络模型。

In [2]:
import mindspore
from mindspore import nn, ops

#mindspore.nn: 提供了构建神经网络的层和组件，类似于torch.nn和tf.keras.layers
#mindspore.ops: 提供了底层操作和函数式API，类似于torch.nn.functional+torch操作和tf.nn+tf.math

## 定义模型类

当我们定义神经网络时，可以继承`nn.Cell`类，在`__init__`方法中进行子Cell的实例化和状态管理，在`construct`方法中实现Tensor操作。

In [18]:
class Network(nn.Cell):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten() 
        #创建一个扁平化层，用于将输入的多维数据（如图像）展平为一维向量，28*28=784
        
        self.dense_relu_sequential = nn.SequentialCell(
            #nn.SequentialCell 是一个容器，按顺序包含多个网络层，数据按顺序通过这些层

            nn.Dense(28*28, 512, weight_init="normal", bias_init="zeros"),
            nn.ReLU(),
            nn.Dense(512, 512, weight_init="normal", bias_init="zeros"),
            nn.ReLU(),
            nn.Dense(512, 10, weight_init="normal", bias_init="zeros")
        )

    def construct(self, x):  
        #construct是MindSpore中定义网络前向传播的方法（等同于PyTorch中的forward
        x = self.flatten(x)
        logits = self.dense_relu_sequential(x)
        
        return logits

构建完成后，实例化`Network`对象，并查看其结构。

In [19]:
model = Network()
print(model)

Network<
  (flatten): Flatten<>
  (dense_relu_sequential): SequentialCell<
    (0): Dense<input_channels=784, output_channels=512, has_bias=True>
    (1): ReLU<>
    (2): Dense<input_channels=512, output_channels=512, has_bias=True>
    (3): ReLU<>
    (4): Dense<input_channels=512, output_channels=10, has_bias=True>
    >
  >


我们构造一个输入数据，直接调用模型，可以获得一个十维的Tensor输出，其包含每个类别的原始预测值。

tips: `model.construct()`方法不可直接调用。
- 使用`model(inputs)`形式调用模型，这会间接调用`construct()`
- 使用MindSpore提供的高级API如`model.train`、`model.predict`等会自动调用`construct()`。

In [20]:
X = ops.ones((1, 28, 28), mindspore.float32)
logits = model(X)
print(logits)

[[ 4.3765442e-03  6.6814530e-03  2.0682917e-03  7.6120538e-03
  -1.0852818e-06 -5.9628091e-03 -5.5858246e-03  1.0529639e-03
   7.6235924e-03 -1.1235978e-03]]


在此基础上，我们通过一个`nn.Softmax`层实例来获得预测概率。

In [21]:
pred_probab = nn.Softmax(axis=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

#训练的时候往往不会把nn.softmax()函数放进construct层，
#一般会选择使用交叉熵损失函数，nn.SoftmaxCrossEntropyWithLogits已经在内部实现了Softmax操作

Predicted class: [8]


## 模型层

我们分解上节神经网络模型中的每一层。

首先我们构造一个shape为(3, 28, 28)的随机数据（3个28x28的图像），依次通过每一个神经网络层来观察其效果。

In [3]:
input_image = ops.ones((3, 28, 28), mindspore.float32)
print(input_image.shape)

(3, 28, 28)


### nn.Flatten

实例化`nn.Flatten` 层，`nn.Flatten()` 将把除了第一个维度（通常是批量大小batch_size）之外的所有维度展平成一个维度。

即，将28x28的2D张量转换为784大小的连续数组。


In [5]:
flatten = nn.Flatten()
#flatten = nn.Flatten(start_dim=2)
flat_image = flatten(input_image)
print(flat_image.shape)

(3, 784)


### nn.Dense

`nn.Dense`为全连接层，使用weights和bias对输入进行线性变换。

In [32]:
layer1 = nn.Dense(in_channels=28*28, out_channels=20)
hidden1 = layer1(flat_image)
print(hidden1.shape)

(3, 20)


### nn.ReLU

`nn.ReLU`层给网络中加入非线性的激活函数，帮助神经网络学习各种复杂的特征。

In [33]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: [[-0.02011143 -0.26672617 -0.81789654 -0.04325772  0.8912562  -0.20275453
   0.16646272 -0.42533413 -0.02069402 -0.40331644 -0.14965856  0.9365725
  -0.82388407  0.25218958  0.6173708   0.18822476  0.57242197 -0.7200695
  -0.28978503  0.06200574]
 [-0.02011143 -0.26672617 -0.81789654 -0.04325772  0.8912562  -0.20275453
   0.16646272 -0.42533413 -0.02069402 -0.40331644 -0.14965856  0.9365725
  -0.82388407  0.25218958  0.6173708   0.18822476  0.57242197 -0.7200695
  -0.28978503  0.06200574]
 [-0.02011143 -0.26672617 -0.81789654 -0.04325772  0.8912562  -0.20275453
   0.16646272 -0.42533413 -0.02069402 -0.40331644 -0.14965856  0.9365725
  -0.82388407  0.25218958  0.6173708   0.18822476  0.57242197 -0.7200695
  -0.28978503  0.06200574]]


After ReLU: [[0.         0.         0.         0.         0.8912562  0.
  0.16646272 0.         0.         0.         0.         0.9365725
  0.         0.25218958 0.6173708  0.18822476 0.57242197 0.
  0.         0.06200574]
 [0.         0.    

### nn.SequentialCell

`nn.SequentialCell`是一个有序的Cell容器。输入Tensor将按照定义的顺序通过所有Cell。

我们可以使用`nn.SequentialCell`来快速组合构造一个神经网络模型。

In [34]:
seq_modules = nn.SequentialCell(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Dense(20, 10)
)

logits = seq_modules(input_image)
print(logits.shape)

(3, 10)


### nn.Softmax

最后使用[nn.Softmax](https://www.mindspore.cn/docs/zh-CN/r2.3/api_python/nn/mindspore.nn.Softmax.html)将神经网络最后一个全连接层返回的logits的值缩放为\[0, 1\]，表示每个类别的预测概率。`axis`指定的维度数值和为1。

In [35]:
softmax = nn.Softmax(axis=1)
pred_probab = softmax(logits)

## 模型参数

网络内部神经网络层具有权重参数和偏置参数（如`nn.Dense`），这些参数会在训练过程中不断进行优化。

可通过 `model.parameters_and_names()` 来获取参数名及对应的参数详情。

In [36]:
print(f"Model structure: {model}\n\n")

for name, param in model.parameters_and_names():
    print(f"Layer: {name}\nSize: {param.shape}\nValues : {param[:2]} \n")

Model structure: Network<
  (flatten): Flatten<>
  (dense_relu_sequential): SequentialCell<
    (0): Dense<input_channels=784, output_channels=512, has_bias=True>
    (1): ReLU<>
    (2): Dense<input_channels=512, output_channels=512, has_bias=True>
    (3): ReLU<>
    (4): Dense<input_channels=512, output_channels=10, has_bias=True>
    >
  >


Layer: dense_relu_sequential.0.weight
Size: (512, 784)
Values : [[ 0.00237562 -0.00517793  0.00126484 ...  0.00091744  0.01295729
  -0.00779298]
 [ 0.01920481 -0.00843089  0.00683935 ... -0.0012791  -0.00246767
  -0.00070447]] 

Layer: dense_relu_sequential.0.bias
Size: (512,)
Values : [0. 0.] 

Layer: dense_relu_sequential.2.weight
Size: (512, 512)
Values : [[-0.01328323  0.00447932 -0.00536488 ... -0.00459299 -0.00993784
  -0.0070957 ]
 [-0.00471527  0.0024026   0.00012553 ...  0.00692865 -0.00701123
   0.00399537]] 

Layer: dense_relu_sequential.2.bias
Size: (512,)
Values : [0. 0.] 

Layer: dense_relu_sequential.4.weight
Size: (10, 512)
Valu